In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.preprocessing import StandardScaler


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample.csv")

In [352]:
def preprocess(df):
  df.index = pd.to_datetime(df["datetime"])
  df = df["2014-05-01":]
  df = df.reset_index(drop=True)
  
  def weather(x):
    if x.find("快晴"):
      x = 0
    elif x.find("晴れ"):
      x = 1
    elif x.find("曇"):
      x = 2
    elif x.find("薄曇"):
      x = 3
    elif x.find("雨"):
      x = 4
    else:
      x = 5
    return x

  df["days"] = df.index
  df["precipitation"] = df["precipitation"].apply(lambda x : -1 if x=="--" else x).astype(np.float64)
  df["fun"] = df["remarks"].apply(lambda x: 1 if x=="お楽しみメニュー" else 0)
  df["curry"] = df["name"].apply(lambda x : 1 if x.find("カレー")>=0 else 0)
  df["weather"] = df["weather"].apply(weather)
  #df["subuta"] = df["name"].apply(lambda x : 1 if x.find("酢豚")>=0 else 0)
  #df["sp"] = df["remarks"].apply(lambda x : 1 if x==("スペシャルメニュー")>=0 else 0)
  #df["kd"] = df["remarks"].apply(lambda x : 1 if x==("料理長")>=0 else 0)
  
  categorical_cols = ["precipitation", "weather", "days", "fun", "curry", "week", "name", "remarks", "event"]
  
  for cols in categorical_cols:
    if cols in df.columns:
      df[cols] = df[cols].astype("category")
  return df

In [353]:
df_train = preprocess(train)
test["y"] = 0
df_test = preprocess(test)
cols = ["datetime","weather","days","fun","curry","y"]


In [ ]:
# ５月１日～９月３０日
df_val = df_train[df_train["datetime"] >="2014-7-28"][cols] # 検証データ
df_tr = df_train[df_train["datetime"] <"2014-7-28" ][cols] # 学習データ

df_val_x = df_val.drop(["y","datetime"],axis=1) # 説明変数の設定
df_tr_x = df_tr.drop(["y","datetime"],axis=1)

df_val["y"] = df_val["y"].astype(int) #エラー修正
df_tr["y"] = df_tr["y"].astype(int) #エラー修正

df_val_y = df_val["y"] # 目的変数の設定
df_tr_y = df_tr["y"]

In [355]:
trains = lgb.Dataset(df_tr_x,df_tr_y)
valids = lgb.Dataset(df_val_x,df_val_y)

In [356]:
fixed_params = {
    "objective":"regression",
    "metric":"mae",
    "min_gain_to_split":0,
    "min_data_in_leaf":0
}

In [357]:
model = lgb.train(fixed_params,trains,valid_sets=valids,num_boost_round=1000,callbacks=[lgb.early_stopping(stopping_rounds=100)])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000073 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12
[LightGBM] [Info] Number of data points in the train set: 54, number of used features: 4
[LightGBM] [Info] Start training from score 68.592593
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [

In [358]:
pred = model.predict(df_test[cols].drop(["y","datetime"],axis=1))
pred

array([ 64.00000016,  47.00000075,  64.36363647,  64.36363647,
        64.36363647, 121.3333315 ,  64.36363647, 121.3333315 ,
        57.00000039,  64.36363647,  64.36363647,  57.00000039,
        64.36363647,  64.36363647,  64.36363647,  64.36363647,
        78.49999964,  64.36363647,  57.00000039,  57.00000039,
        64.36363647,  64.36363647,  64.36363647,  64.36363647,
        64.36363647, 121.3333315 ,  57.00000039,  64.36363647,
        64.36363647,  57.00000039,  57.00000039,  64.36363647,
        57.00000039,  57.00000039,  64.36363647,  78.49999964,
        64.36363647,  64.36363647,  57.00000039,  64.36363647])

In [ ]:
import optuna
from optuna.integration import LightGBMPruningCallback

In [360]:
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'num_leaves': trial.suggest_int('num_leaves', 10, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'verbose': -1,
        'random_state': 42
    }
    
    #検証
    cv_scores = []
    df_val = df_train[df_train["datetime"] >="2014-7-28"][cols] 
    df_tr = df_train[df_train["datetime"] <"2014-7-28" ][cols]
    df_val_x = df_val.drop(["y","datetime"],axis=1) # 説明変数の設定
    df_tr_x = df_tr.drop(["y","datetime"],axis=1)

    df_val["y"] = df_val["y"].astype(int) #エラー修正
    df_tr["y"] = df_tr["y"].astype(int) #エラー修正

    df_val_y = df_val["y"] # 目的変数の設定
    df_tr_y = df_tr["y"]
    
    trains = lgb.Dataset(df_tr_x,df_tr_y)
    valids = lgb.Dataset(df_val_x,df_val_y)
    
    pruning_callback = LightGBMPruningCallback(trial,"mae")
    model = lgb.train(fixed_params,trains,valid_sets=valids,num_boost_round=1000,callbacks=[lgb.early_stopping(stopping_rounds=100)])
    
    pred = model.predict(df_test[cols].drop(["y","datetime"],axis=1))
    
    
    

In [361]:
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=42)
    )
study.optimize(objective,n_trials=100,show_progress_bar=False)

[I 2025-11-10 12:21:52,063] A new study created in memory with name: no-name-d0483c94-f650-4a17-b06e-425ee01c37b6
[W 2025-11-10 12:21:52,144] Trial 0 failed with parameters: {'num_leaves': 118, 'learning_rate': 0.2536999076681772, 'feature_fraction': 0.839196365086843, 'bagging_fraction': 0.759195090518222, 'bagging_freq': 2, 'min_child_samples': 19, 'lambda_l1': 3.3323645788192616e-08, 'lambda_l2': 0.6245760287469893} because of the following error: The value None could not be cast to float..


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000064 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12
[LightGBM] [Info] Number of data points in the train set: 54, number of used features: 4
[LightGBM] [Info] Start training from score 68.592593
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [

[W 2025-11-10 12:21:52,145] Trial 0 failed with value None.
[W 2025-11-10 12:21:52,211] Trial 1 failed with parameters: {'num_leaves': 184, 'learning_rate': 0.11114989443094977, 'feature_fraction': 0.41235069657748147, 'bagging_fraction': 0.9819459112971965, 'bagging_freq': 6, 'min_child_samples': 25, 'lambda_l1': 4.329370014459266e-07, 'lambda_l2': 4.4734294104626844e-07} because of the following error: The value None could not be cast to float..
[W 2025-11-10 12:21:52,211] Trial 1 failed with value None.
[W 2025-11-10 12:21:52,272] Trial 2 failed with parameters: {'num_leaves': 98, 'learning_rate': 0.05958389350068958, 'feature_fraction': 0.6591670111852694, 'bagging_fraction': 0.5747374841188252, 'bagging_freq': 5, 'min_child_samples': 18, 'lambda_l1': 4.258943089524393e-06, 'lambda_l2': 1.9826980964985924e-05} because of the following error: The value None could not be cast to float..
[W 2025-11-10 12:21:52,272] Trial 2 failed with value None.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000047 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12
[LightGBM] [Info] Number of data points in the train set: 54, number of used features: 4
[LightGBM] [Info] Start training from score 68.592593
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [

[W 2025-11-10 12:21:52,334] Trial 3 failed with parameters: {'num_leaves': 142, 'learning_rate': 0.14447746112718687, 'feature_fraction': 0.5198042692950159, 'bagging_fraction': 0.708540663048167, 'bagging_freq': 5, 'min_child_samples': 9, 'lambda_l1': 0.0029369981104377003, 'lambda_l2': 3.425445902633376e-07} because of the following error: The value None could not be cast to float..
[W 2025-11-10 12:21:52,335] Trial 3 failed with value None.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000052 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12
[LightGBM] [Info] Number of data points in the train set: 54, number of used features: 4
[LightGBM] [Info] Start training from score 68.592593
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [

[W 2025-11-10 12:21:59,327] Trial 96 failed with parameters: {'num_leaves': 109, 'learning_rate': 0.163341530402562, 'feature_fraction': 0.46637904217312437, 'bagging_fraction': 0.9078713750407109, 'bagging_freq': 1, 'min_child_samples': 43, 'lambda_l1': 0.149850532999748, 'lambda_l2': 2.2348935706722892e-07} because of the following error: The value None could not be cast to float..
[W 2025-11-10 12:21:59,328] Trial 96 failed with value None.
[W 2025-11-10 12:21:59,393] Trial 97 failed with parameters: {'num_leaves': 76, 'learning_rate': 0.11664190932505923, 'feature_fraction': 0.8320219219276446, 'bagging_fraction': 0.7846885797311784, 'bagging_freq': 5, 'min_child_samples': 57, 'lambda_l1': 1.8458292771752905e-06, 'lambda_l2': 1.2920042478529478e-05} because of the following error: The value None could not be cast to float..
[W 2025-11-10 12:21:59,394] Trial 97 failed with value None.
[W 2025-11-10 12:21:59,452] Trial 98 failed with parameters: {'num_leaves': 62, 'learning_rate': 0.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[W 2025-11-10 12:21:59,514] Trial 99 failed with parameters: {'num_leaves': 157, 'learning_rate': 0.08001880244016521, 'feature_fraction': 0.4108661102925043, 'bagging_fraction': 0.9232743453664909, 'bagging_freq': 7, 'min_child_samples': 59, 'lambda_l1': 0.018614758870924415, 'lambda_l2': 2.0067553551675026} because of the following error: The value None could not be cast to float..
[W 2025-11-10 12:21:59,514] Trial 99 failed with value None.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f